In [1]:
from CU_Dataset_Factory import Hf_Loader, CU_Dataset_Factory

/home/andrea/miniconda3/envs/MNLP/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
factory = CU_Dataset_Factory('.')
train_l = Hf_Loader("sapienzanlp/nlp2025_hw1_cultural_dataset", 'train')
validation_l = Hf_Loader("sapienzanlp/nlp2025_hw1_cultural_dataset", 'validation')

In [3]:
fe = 'intro'
model_repo = 'google/mobilebert-uncased'

In [4]:
train = factory.produce(train_l, 'tr_train.tsv', [fe], 'label', 32, False)
test  = factory.produce(validation_l, 'tr_validation.tsv', [fe], 'label', 32, False)

intro: 100%|██████████| 300/300 [00:00<00:00, 9856.43it/s, batch=10] 


In [5]:
train_data = train[['label', fe]].rename({fe: 'text'}, axis=1)
validation_data = test[['label', fe]].rename({fe: 'text'}, axis=1)

In [6]:
train.head(50)

,intro,wiki_name,qid,label
0,,916 (film),Q32786,1
1,"!!! ( ch(i)k-ch(i)k-ch(i)k), also known as Chk...",!!!,Q371,2
2,,¡Soborno!,Q3729947,2
3,+44 (read as Plus Forty-four) was an American ...,+44 (band),Q158611,2
4,"1 Monk Street, Monmouth was built as a Working...","1 Monk Street, Monmouth",Q280375,1
5,The 100 percent corner is the busiest area in ...,100 percent corner,Q104414508,0
6,The 1889 Apia cyclone was a tropical cyclone i...,1889 Apia cyclone,Q3008535,0
7,18th Abduction is the eighteenth novel in the ...,18th Abduction,Q85720601,0
8,The 20 July plot was a failed attempt to assas...,20 July plot,Q105570,1
9,"20th Century Studios, Inc., formerly 20th Cent...",20th Century Studios,Q434841,2


In [7]:
from datasets import Dataset
from datasets import Features
from datasets import Split, Value

In [8]:
train_data = Dataset.from_pandas(train_data, features=Features({
    'label': Value('int32'),
    'text' : Value('string')
}), split=Split.TRAIN)

In [9]:
validation_data = Dataset.from_pandas(validation_data, features=Features({
    'label': Value('int32'),
    'text' : Value('string')
}), split=Split.TEST)

In [10]:
from transformers import AutoTokenizer

In [11]:
tokenizer = AutoTokenizer.from_pretrained('google/mobilebert-uncased')

In [12]:
class Preprocessor:
    def __init__(self, tokenizer) -> None:
        self.tokenizer = tokenizer
    def process_samples(self, samples):
        return samples.map(lambda sample: self.tokenizer(sample['text'], truncation=True))

In [13]:
p = Preprocessor(tokenizer)

In [14]:
tokenize_train = p.process_samples(train_data)
tokenize_test = p.process_samples(validation_data)

Map: 100%|██████████| 300/300 [00:00<00:00, 2919.10 examples/s]


In [15]:
from transformers import DataCollatorWithPadding

In [16]:
collector = DataCollatorWithPadding(tokenizer)

In [17]:
import numpy as np
import evaluate

In [18]:
def compute_metrics(eval_pred):
    acc = evaluate.load('accuracy')
    
    
    pred, true = eval_pred
    pred = np.argmax(pred, axis=1)
    return acc.compute(predictions=pred, references=true)

In [19]:
cls2label = {0:'Cultural Agnostic', 1:'Cultural Rapresentative', 2:'Cultural Exclusive'}
label2cls = {l:c for c ,l in cls2label.items()}

In [20]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

In [21]:
model = model = AutoModelForSequenceClassification.from_pretrained(model_repo, num_labels=3, ignore_mismatched_sizes=True)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
traning_args = TrainingArguments(
    output_dir='CU_with_bert',
    eval_strategy='epoch',
    push_to_hub=False,
    num_train_epochs = 10,
    
)

trainer = Trainer(
    model=model,
    args=traning_args,
    data_collator=collector,
    train_dataset=tokenize_train,
    eval_dataset=tokenize_test,
    compute_metrics=compute_metrics,
)

In [23]:
trainer.model.device

device(type='cuda', index=0)

In [24]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pizzi-1995517 (pizzi-1995517-sapienza-universit-di-roma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


RuntimeError: The size of tensor a (661) must match the size of tensor b (512) at non-singleton dimension 1